# FDA Drug Repurposing - Join and Load to CAS

0) run this AFTER DrugBank & FDA Clinical Trials python processes

1) set up envrionment
- if download directories match Box - this program should run without modification
- to enable CAS table load:
    - edit Cell 3 "casenabled" to true
    - modify CAS server to your address
    - modify CAS ID to your ID
    - and add your password

2) Read in dimensions for drug repositioning
<br><br>
- DrugBank: Drug (pre processed by DrugBank_drug_target_relationship_v2.ipynb)
dbSmallM_df = pd.read_csv(DBPath + 'DrugBank_CSVs/drugbank-slim.tsv', sep='\t')
dbIndication_df = pd.read_csv(DBPath + 'DrugBank_CSVs/drugbank-indication.tsv', sep='\t')
- DrugBank: Drug/Protein
dbProteins_df = pd.read_csv(DBPath + 'DrugBank_CSVs/proteins.tsv', sep='\t')
- DrugBank: synonyms
fObj = open(DBPath+'DrugBank_CSVs/aliases.json',)

<br><br>
- ClinicalTrials: COVID Truth (pre processed by FDA_Read_ClinicalTrialsTruth.ipynb)
Truth_df = pd.read_csv(CTPath + 'ProposedTruth.csv')
truthFinal_df = pd.read_csv(CASPublicPath + 'truthfinal.csv') - from PUBLIC CASLIB

<br><br>
- UniProt: Protein synonym IDs for joining Drugbank, String and VirusHost proteins<br>
with open(UniProtPath + 'HUMAN_9606_idmapping.dat') as f:
    row_count = sum(1 for row in f)
display(row_count)

    source: https://ftp.uniprot.org/pub/databases/uniprot/current_release/knowledgebase/idmapping/by_organism/
<br><br>
- VirusHost: SARS Related Virus to Host Protein
    - VirusHostProt_df = pd.read_csv(VHProtPath + 'HCoV-associated host proteins.csv')
- String: Protein-Protein
    - stringPP_df = pd.read_csv(STRINGPath + '9606.protein.links.v11.0.txt', sep=' ')
<br>

3) write out base tables to CASLIB + CSVs
- UniProt Lookups
    - UniProtGeneIDs_df.to_csv(CSVOut+"/"+"UniProtGeneIDs.csv")
    - UniProtGeneIDs_ct = s.upload_file(data=UniProtGeneIDs_df, casout=dict(name='UniProtGeneIDs', caslib='public', replication=0, promote=True))
    
    - UniProtSTRINGIDs_df.to_csv(CSVOut+"/"+"UniProtSTRINGIDs.csv")
    - UniProtSTRINGIDs_ct = s.upload_file(data=UniProtSTRINGIDs_df, casout=dict(name='UniProtSTRINGIDs', caslib='public', replication=0, promote=True))
    
    - UniProtGeneNames_df.to_csv(CSVOut+"/"+"UniProtGeneNames.csv")
    - UniProtGeneNames_ct = s.upload_file(data=UniProtGeneNames_df, casout=dict(name='UniProtGeneNames', caslib='public', replication=0, promote=True))

- Truth Sources
    - Truth_df.to_csv(CSVOut+"/"+"Truth.csv")
    - Truth_ct = s.upload_file(data=Truth_df, casout=dict(name='Truth', caslib='public', replication=0, promote=True, label="Clinical Trials Truth: NOTE needs to be refined"))

    - truthMatch2_df.to_csv(CSVOut+"/"+"newTruth.csv")
    - newTruth_ct = s.upload_file(data=truthMatch2_df, casout=dict(name='newTruth', caslib='public', replication=0, promote=True, label="Clinical Trials Truth: NOTE needs to be refined"))

- String mapped to UniProt so it will match DrugBank Drug-Protein IDs
    - stringPP_ct = s.upload_file(data=stringPP_df, casout=dict(name='stringPP', caslib='public', replication=0, promote=True, label="STRING Protein Protein interactions NOTE Missing UniProt Matches"))
    - stringPP_df.to_csv(CSVOut+"/"+"stringPP.csv")

- Virus Host Protein mapped to UniProt so it will match DrugBank Drug-Protein IDs
    - VirusHostProt_df.to_csv(CSVOut+"/"+"VirusHostProt.csv")
    - VirusHostProt_ct = s.upload_file(data=VirusHostProt_df, casout=dict(name='VirusHostProt', caslib='public', replication=0, promote=True, label="Virus Host Protein Relationships NOTE: Many Proteins had 1+ UniProt IDs"))

- DrugBank_IDs mapped to both "truth" tables
    - x.to_csv(CSVOut+"/"+"dbSmallM_Truth.csv")
    - dbSmallM_ct = s.upload_file(data=x, casout=dict(name='dbSmallM_Truth', caslib='public', replication=0, promote=True, label="Drug Bank: Small Molecule drugs (NOTE additional attributes available plus stings need to be parsed)"))

- Drugbank Drug-Proteins
    - dbProteins_df.to_csv(CSVOut+"/"+"dbProteins.csv")
    - dbProteins_ct = s.upload_file(data=dbProteins_df, casout=dict(name='dbProteins', caslib='public', replication=0, promote=True, label="Drug Bank: Small Molcule related proteins (NOTE not all captured by XML parser yet)"))


## 1) setup environment

In [ ]:
# load packages

import pandas as pd
import numpy as np
import os

import time
import getpass


In [ ]:
# files / paths
print(os.getcwd())
FDAPath = os.getcwd() + '/../data/source/'

CTPath = FDAPath + 'ClinicalTrials/'
DBPath = FDAPath + "DrugBank/"
STRINGPath = FDAPath + 'STRING/'
VHProtPath = FDAPath + 'VirusHostProteins/'
UniProtPath = FDAPath + 'UniProt/'

# CASPublicPath = '/opt/sas/viya/config/data/cas/default/public/'   # truthfinal.csv exported from cas
CSVOut = FDAPath + '../DrugRepositioning'

print(os.getcwd())
os.chdir(FDAPath)
print(os.getcwd())




In [ ]:
# link to CAS
# https://developer.sas.com/guides/python-swat.html

casenabled = False
casServerURL = "pdcesx06019.exnet.sas.com"

casPort = 5570
sasid = 'sasdemo'
saspw = 'xxxxx'
if casenabled == True:
    saspw = '<set your pw here>' #getpass.getpass() 
    import swat
    s = swat.CAS(casServerURL, casPort, sasid, saspw)
    #s = swat.CAS("pdcesx15065.exnet.sas.com", 5570, 'sasdemo', authinfo='/home/sasdemo/.authinfo')
    print("CAS is enabled")
else:
    print("No CAS")

In [ ]:
if casenabled == True:
    s.serverstatus()


## 2) read base tables

In [ ]:
# DrugBank (this is a database with multiple tables)
#  pre-processed in drug_target_relationship_DrugBank_rkc.ipynb
#  note: needs more work (as of 2021 02 18)

# drug bank - same info as "Slim" but all IDs instead of just small molecule
#db_df = pd.read_csv(DBPath + 'drugbank.tsv', sep='\t')
#display(db_df.head())

# drug bank - Small Molecule IDs only
dbSmallM_df = pd.read_csv(DBPath + 'DrugBank_CSVs/drugbank-slim.tsv', sep='\t')
dbIndication_df = pd.read_csv(DBPath + 'DrugBank_CSVs/drugbank-indication.tsv', sep='\t')
display(dbSmallM_df.shape)
dbSmallM_df = dbSmallM_df.merge(dbIndication_df[['drugbank_id', 'indication']], on='drugbank_id', how='left')
display(dbSmallM_df.shape)
display(dbSmallM_df.head())
del dbIndication_df

dbProteins_df = pd.read_csv(DBPath + 'DrugBank_CSVs/proteins.tsv', sep='\t')
dbProteins_df = dbProteins_df.merge(dbSmallM_df[['drugbank_id']], on='drugbank_id', how='inner')
display(dbProteins_df.shape)
display(dbProteins_df.head())


In [ ]:
# FDA Truth - list of DrugBank Drug IDs that should be considered the "right" answers
#  pre-processed in FDA_Read_ClinicalTrialsTruth.ipynb
#  note: needs more work (as of 2021 02 18)
Truth_df = pd.read_csv(CTPath + 'ProposedTruth.csv')
display(Truth_df.head())

Truth_df.rename(columns = {'iDBID':'drugbank_id'}, inplace = True)
Truth_df['truth'] = 1
Truth_df = Truth_df[['drugbank_id', 'truth', 'iDBIDCount', 'iName', 'inputRowCount', 'inputRowIDs']]
display(Truth_df.shape)
display(Truth_df.head())


In [ ]:
# Truth Final - from (?) inn CAS Public folder
# probably could read directly from PUBLIC caslib

#####  NOTE !!!   This came AFTER the above table - was imported to CAS 
#####               a new source was appended
#####               and now this replaces the above in the process

truthFinal_df = pd.read_csv(CTPath + 'truthfinal.csv')
display(truthFinal_df.shape)
display(truthFinal_df.head(20))


In [ ]:
# match CASLIB truthFinal to Drugbank synonyms json
# (extracted code from FDA_Read_ClincalTrialsTruth.ipynb)


import json

fObj = open(DBPath+'DrugBank_CSVs/aliases.json',)
DBAlias = json.load(fObj)
fObj.close()

print(DBAlias.__class__)
display(DBAlias['DB12466'])
display(DBAlias["DB15327"])

# search dict in DBAlias format to return ID if element of an array matches
def searchDBAlias(byVal):
    keysList = []
    itemsList = DBAlias.items()
    for item in itemsList:
        if byVal in item[1]:
            keysList.append(item[0])
    return keysList

# this could be sped up if it becomes an issue
start = time.time()
truthFinal_df['Trail_Drug_DBID'] = truthFinal_df['Trail_Drug'].apply(searchDBAlias)
lapse = time.time() - start 
print("lapse time to match: ", lapse)


In [ ]:
# count Drug Bank ID matches
truthFinal_df['iDBIDCount'] = truthFinal_df['Trail_Drug_DBID'].astype('str').str.count("'")/2 
display(truthFinal_df.shape)
display(truthFinal_df.head(20))


In [ ]:
display(truthFinal_df.shape)
display(truthFinal_df.head(20))

#display(truthFinal_df.Trail_Drug_DBID.apply(len)==0)
#display(truthFinal_df.NewDrug==1)
#display(np.logical_and(truthFinal_df.NewDrug==1, truthFinal_df.Trail_Drug_DBID.apply(len)==0))

#display(truthFinal_df[np.logical_and((truthFinal_df.NewDrug==0), (truthFinal_df.Trail_Drug_DBID.apply(len)!=0))].shape)
#display(truthFinal_df[np.logical_and((truthFinal_df.NewDrug==0), (truthFinal_df.Trail_Drug_DBID.apply(len)!=0))].head(50))

display(truthFinal_df[np.logical_and((truthFinal_df.NewDrug==0), (truthFinal_df.iDBIDCount!=0))].shape)
display(truthFinal_df[np.logical_and((truthFinal_df.NewDrug==0), (truthFinal_df.iDBIDCount!=0))].head(50))

display(truthFinal_df[np.logical_and((truthFinal_df.NewDrug==1), (truthFinal_df.iDBIDCount!=0))].shape)
display(truthFinal_df[np.logical_and((truthFinal_df.NewDrug==1), (truthFinal_df.iDBIDCount!=0))].head(50))

display(truthFinal_df[np.logical_and((truthFinal_df.NewDrug==1), (truthFinal_df.iDBIDCount==0))].shape)
display(truthFinal_df[np.logical_and((truthFinal_df.NewDrug==1), (truthFinal_df.iDBIDCount==0))].head(200))


In [ ]:
truthMatch_df = truthFinal_df[truthFinal_df.iDBIDCount!=0]
#display(truthMatch_df.shape)
#display(truthMatch_df.head(40))
display(truthMatch_df['NewDrug'].value_counts())
display(truthMatch_df['iDBIDCount'].value_counts())

tmids = truthMatch_df.apply(lambda x: pd.Series(x['Trail_Drug_DBID']),axis=1).stack().reset_index(level=1, drop=True)
tmids.name = 'iDBID'
#display(tmids)

truthMatch_df = truthMatch_df.drop(['Trail_Drug_DBID', 'iDBIDCount'], axis=1).join(tmids)
display(truthMatch_df.shape)
display(truthMatch_df[10:30])


In [ ]:
truthMatch_df.drop_duplicates(subset='iDBID', keep='first', inplace=True)
display(truthMatch_df.shape)
display(truthMatch_df[10:30])
display(truthMatch_df['NewDrug'].value_counts())


In [ ]:
# flag if DB ID has proteins associated or not
display(dbProteins_df.shape)
display(dbProteins_df.head())

# count proteins per DBID
proteinCount_df = pd.DataFrame(dbProteins_df.drugbank_id.value_counts().reset_index().values, columns=["iDBID", "ProteinCount"])
proteinCount_df.sort_values(by=['iDBID'], inplace=True)
proteinCount_df.reset_index(drop=True, inplace=True)
display(proteinCount_df.shape)
display(proteinCount_df.head())

display(truthMatch_df.shape)
display(truthMatch_df.head())

# merge count in
truthMatch2_df = truthMatch_df.merge(proteinCount_df, how="left", on='iDBID').fillna(0)
display(truthMatch2_df.shape)
display(truthMatch2_df.head())


In [ ]:
# merge fda truth in to validate new/old status

# first dedupe old Truth_df
display(Truth_df.shape)
Truth_df.drop_duplicates(subset='drugbank_id', keep='first', inplace=True)
display(Truth_df.shape)
display(Truth_df[['drugbank_id', 'truth']].head())

truthMatch2_df = truthMatch2_df.merge(Truth_df[['drugbank_id', 'truth']], how="left", left_on='iDBID', right_on='drugbank_id').fillna(0)
display(truthMatch2_df.shape)
display(truthMatch2_df.head())

display(pd.crosstab(truthMatch2_df.truth, truthMatch2_df.NewDrug, margins=True))

# fix incorrect NewDrug flags
truthMatch2_df.NewDrug[truthMatch2_df.truth==1] = 0
display(pd.crosstab(truthMatch2_df.truth, truthMatch2_df.NewDrug, margins=True))


In [ ]:
display(truthMatch2_df.shape)
display(truthMatch2_df.head())
display(pd.crosstab(truthMatch2_df.ProteinCount>0, truthMatch2_df.NewDrug, margins=True))


In [ ]:
# UniProt - protein IDs / synonyms  ( to merge in with STRING and Virus/Protein data so they can merge with DrugBank)

# exclude UniProt IDs beginning with A0A per this reference:
# https://ftp.uniprot.org/pub/databases/uniprot/knowledgebase/docs/sec%5Fac.txt

with open(UniProtPath + 'HUMAN_9606_idmapping.dat') as f:
    row_count = sum(1 for row in f)
display(row_count)

# can I read all of just column 1 ?
# if yes - I can use that to create an array of only STRING, only Gene_Name and only GeneID 
#    and then just read columns 0 & 2 from THAT subset
# else if that runs into space limits - just use GREP to extract the rows we want !
#    grep Gene_Name *.dat
altIDNames_df = pd.read_csv(UniProtPath + 'HUMAN_9606_idmapping.dat', usecols = [1], sep='\t', header=None)
altIDNames_df.columns = ['altIDName']
display(altIDNames_df.shape)
display(altIDNames_df.head())
display(altIDNames_df.altIDName.value_counts())

STRINGSkip = altIDNames_df.index[altIDNames_df.altIDName != 'STRING']
EnsemblProSkip = altIDNames_df.index[altIDNames_df.altIDName != 'Ensembl_PRO']
GNameSkip = altIDNames_df.index[altIDNames_df.altIDName != 'Gene_Name']
GIDSkip = altIDNames_df.index[altIDNames_df.altIDName != 'GeneID']

del altIDNames_df


#-------------------
# STRING ID matching
display(STRINGSkip.shape)
UniProtSTRINGIDs_df = pd.read_csv(UniProtPath + 'HUMAN_9606_idmapping.dat', usecols=[0,2], skiprows=set(STRINGSkip), sep='\t', header=None)
UniProtSTRINGIDs_df.columns = ['UniProtID', 'STRINGID']
display(UniProtSTRINGIDs_df.shape)
display(UniProtSTRINGIDs_df.head())

#-------------------
# Ensembl Pro matching
display(EnsemblProSkip.shape)
UniProtEnsemblePro_df = pd.read_csv(UniProtPath + 'HUMAN_9606_idmapping.dat', usecols=[0,2], skiprows=set(EnsemblProSkip), sep='\t', header=None)
UniProtEnsemblePro_df.columns = ['UniProtID', 'Ensembl_PRO']
UniProtEnsemblePro_df['Ensembl_PRO9606'] = '9606.'+UniProtEnsemblePro_df['Ensembl_PRO']
display(UniProtEnsemblePro_df.shape)
display(UniProtEnsemblePro_df.head())

#-------------------
# Gene ID matching
display(GIDSkip.shape)
UniProtGeneIDs_df = pd.read_csv(UniProtPath + 'HUMAN_9606_idmapping.dat', usecols=[0,2], skiprows=set(GIDSkip), sep='\t', header=None)
UniProtGeneIDs_df.columns = ['UniProtID', 'GeneID']

# strip out duplicate UniProt GeneID rows - or should we be setting an order preference
UniProtGeneIDs_df['FirstChar'] = UniProtGeneIDs_df['UniProtID'].str.slice(0,1)
UniProtGeneIDs_df['A0A_Type'] = (UniProtGeneIDs_df['UniProtID'].str.slice(0,3) == 'A0A')
UniProtGeneIDs_df['ID_Length'] = UniProtGeneIDs_df['UniProtID'].str.len()
cols = ['FirstChar', 'A0A_Type', 'ID_Length']
display(UniProtGeneIDs_df.groupby(cols)[cols].count())

#UniProtGeneIDs_df = UniProtGeneIDs_df[UniProtGeneIDs_df['ID_Length'] == 6]
UniProtGeneIDs_df = UniProtGeneIDs_df[UniProtGeneIDs_df['A0A_Type'] == False]
#UniProtGeneIDs_df = UniProtGeneIDs_df[UniProtGeneIDs_df['FirstChar'].isin(['O', 'P', 'Q'])]
UniProtGeneIDs_df.drop(columns=['FirstChar', 'A0A_Type', 'ID_Length'], inplace=True)

# if > 1 UniProt row for a Host_Gene_ID, then keep P, then Q, then first of any remaining UniProt row(s)
GeneRows = UniProtGeneIDs_df.GeneID.value_counts()
print('The gene IDs with the top 25 most protein counts: ', GeneRows[:25])

display(UniProtGeneIDs_df.shape)
display(UniProtGeneIDs_df.head())

#-------------------
# Gene Name matching
display(GNameSkip.shape)
UniProtGeneNames_df = pd.read_csv(UniProtPath + 'HUMAN_9606_idmapping.dat', usecols=[0,2], skiprows=set(GNameSkip), sep='\t', header=None)
UniProtGeneNames_df.columns = ['UniProtID', 'GeneName']
display(UniProtGeneNames_df.shape)
display(UniProtGeneNames_df.head())

# will probably need to repeat gene-id cleansing steps above if using gene name to match




In [ ]:
# example of a gene with 29 proteins (4 of the A0A type)
# https://worldwide.promega.com/FindMyGene/GeneDetail.aspx?ncbiid=7404
display(UniProtGeneIDs_df[UniProtGeneIDs_df.GeneID == 7404])


In [ ]:
# VirusHost - virus / protein
VirusHostProt_df = pd.read_csv(VHProtPath + 'HCoV-associated host proteins.csv')
display(VirusHostProt_df.shape)
display(VirusHostProt_df.head())

# now get UniProtID merged in on Gene_ID (match on Host_Protein should be equivalent)
#display(UniProtGeneIDs_df.head())

VirusHostProt_df = pd.merge(VirusHostProt_df, UniProtGeneIDs_df, how="left", left_on='Host_Gene_ID', right_on='GeneID', indicator=True)
print("after match (expecting both=130, right/left only = 0): ")
display(VirusHostProt_df['_merge'].value_counts())
VirusHostProt_df.drop(columns=['_merge', 'GeneID'], inplace=True)
display(VirusHostProt_df.shape)
display(VirusHostProt_df.head())


In [ ]:
#display(VirusHostProt_df.iloc[[0,1,2,4,5]])
display(VirusHostProt_df.iloc[:200])


<b>Duplicates exist even after elminating A0A type matches</b><br>
<br>
one example is gene_id 2932 = P49841 and Q6FI27<br>
<br>
https://www.uniprot.org/uniprot/?query=yourlist:M202103028471C63D39733769F8E060B506551E12176354Y&sort=yourlist:M202103028471C63D39733769F8E060B506551E12176354Y&columns=yourlist(M202103028471C63D39733769F8E060B506551E12176354Y),id,entry%20name,reviewed,protein%20names,genes,organism,length


In [ ]:
# STRING - protein/protein

# NOTE - each pair is in the file 2x so only UniProtID1 needs to be matched
# Example below:
#[sasdemo@sasserver STRING]$ grep ENSP00000000233 *.txt | grep -e ENSP00000272298
#9606.ENSP00000000233 9606.ENSP00000272298 490
#9606.ENSP00000272298 9606.ENSP00000000233 490

stringPP_df = pd.read_csv(STRINGPath + '9606.protein.links.v11.0.txt', sep=' ')
display(stringPP_df.shape)
display(stringPP_df.head())

# merge Protein 1 to STRING on UniProt .dat file
stringPP_df = pd.merge(stringPP_df, UniProtSTRINGIDs_df, how="left", left_on='protein1', right_on='STRINGID', indicator=True)
#display(stringPP_df['_merge'].value_counts())
stringPP_df.rename(columns = {'UniProtID':'UniProtID1', 'STRINGID':'STRINGID1'}, inplace = True)
stringPP_df.drop(columns=['_merge'], inplace=True)

# merge Protein 1 to Ensembl_PRO on UniProt .dat file
stringPP_df = pd.merge(stringPP_df, UniProtEnsemblePro_df[['UniProtID', 'Ensembl_PRO9606']], how="left", left_on='protein1', right_on='Ensembl_PRO9606', indicator=True)
#display(stringPP_df['_merge'].value_counts())

# coalesce Protein 1 STRING and Ensemble_PRO columns (use Ensembl_PRO if STRING is missing)
stringPP_df['UniProtID1'] = stringPP_df['UniProtID1'].mask(pd.isnull, stringPP_df['UniProtID'])
stringPP_df.rename(columns = {'Ensembl_PRO9606':'Ensembl_PRO96061'}, inplace = True)
stringPP_df.drop(columns=['_merge', 'UniProtID'], inplace=True)

# merge protein 2 to STRING on UniProt .dat file
stringPP_df = pd.merge(stringPP_df, UniProtSTRINGIDs_df, how="left", left_on='protein2', right_on='STRINGID', indicator=True)
#display(stringPP_df['_merge'].value_counts())
stringPP_df.rename(columns = {'UniProtID':'UniProtID2', 'STRINGID':'STRINGID2'}, inplace = True)
stringPP_df.drop(columns=['_merge'], inplace=True)

# merge Protein 2 to Ensembl_PRO on UniProt .dat file
stringPP_df = pd.merge(stringPP_df, UniProtEnsemblePro_df[['UniProtID', 'Ensembl_PRO9606']], how="left", left_on='protein2', right_on='Ensembl_PRO9606', indicator=True)

# coalesce Protein 2 STRING and Ensemble_PRO columns (use Ensembl_PRO if STRING is missing)
stringPP_df['UniProtID2'] = stringPP_df['UniProtID2'].mask(pd.isnull, stringPP_df['UniProtID'])
stringPP_df.rename(columns = {'Ensembl_PRO9606':'Ensembl_PRO96062'}, inplace = True)
stringPP_df.drop(columns=['_merge', 'UniProtID'], inplace=True)

print()
print('After matching to STRING and Ensembl_PRO in UniProt')
display(stringPP_df.shape)
print("Rows with no match for STRING: ", stringPP_df.STRINGID1.isnull().count())
print("Rows with no match for Ensebml_PRO: ", stringPP_df.Ensembl_PRO96061.isnull().count())
print("Rows with no match for either: ", stringPP_df.UniProtID1.isnull().count())
#print("Rows with no match for either: ", stringPP_df.UniProtID1.isnull().value_counts())
print("NOTE: next most likely reason for missing is depricated IDs.  see note in markup cell below")
print()
display(stringPP_df.head())

print()
print('Sample of Null matches:')
display(stringPP_df[stringPP_df.UniProtID1.isnull()].head())
#display(stringPP_df[stringPP_df.UniProtID1.isnull()].groupby('protein1')['protein1'].count())
print("unique IDs with no match: ", stringPP_df[stringPP_df.UniProtID1.isnull()].groupby('protein1')['protein1'].count().count())

# reset dataframe to contain only relevant columns
stringPP_df = stringPP_df[['protein1', 'UniProtID1', 'protein2', 'UniProtID2', 'combined_score']]

<b>searching UniProt ID Mapping.dat file for unmatched STRING IDs(without 9606 prefix
gives no STRING IDs, but Ensembl_PRO</b>

[sasdemo@sasserver UniProt]$ ls -ltr<br>
-rw-r--r-- 1 cas sas 159854967 Jan 26 14:20 HUMAN_9606_idmapping.dat<br>

[sasdemo@sasserver UniProt]$ grep -e "ENSP00000016946" *.dat<br>
Q99666-1        Ensembl_PRO     ENSP00000016946<br>

[sasdemo@sasserver UniProt]$ grep -e "ENSP00000244537" *.dat<br>
P62805  Ensembl_PRO     ENSP00000244537<br>

[sasdemo@sasserver UniProt]$ grep -e "ENSP00000244537" *.dat<br>
P62805  Ensembl_PRO     ENSP00000244537<br>

<b>searching UniProt ID Mapping.dat for a matched STRINGID (without 9606 prefix)
gives both</b>
[sasdemo@sasserver UniProt]$ grep -e "ENSP00000380178" *.dat<br>
P62253  STRING  9606.ENSP00000380178<br>
P62253  Ensembl_PRO     ENSP00000380178<br>

<b><i>conclusion --> strip out 9606. and try matching to enesmbl_pro.  if that does not work, match STRING first, then match missing to Ensembl_PRO</b></i>


<b>ENSP #s that do not match either STRING or Ensembl_PRO may be depricated IDs such as this one:</b><br>
9606.ENSP00000195455 --> ENST00000195455<br>
9606.ENSP00000377806 --> ENST00000377806<br>
<br>
https://useast.ensembl.org/Homo_sapiens/Transcript/Idhistory/Protein?t=ENSP00000195455
https://useast.ensembl.org/Homo_sapiens/Transcript/Idhistory/Protein?t=ENSP00000377806


## 3) load base to CAS

In [ ]:
if casenabled == True:
    #s.fileinfo(caslib='public') # files have been saved to caslib table backing store
    s.tableinfo(caslib='public') # tables are loaded in memory (and may not have been saved)


In [ ]:
# NOTE: upload_file() can upload disk files into CAS without using local dataframe memory


In [ ]:
# UniProt - select rows for matching:

if casenabled == True:
    s.droptable(caslib='public', name='UniProtGeneIDs', quiet=True)
    UniProtGeneIDs_ct = s.upload_file(data=UniProtGeneIDs_df, casout=dict(name='UniProtGeneIDs', caslib='public', replication=0, promote=True))
    display(UniProtGeneIDs_ct.info())
    display(UniProtGeneIDs_ct.head())

    s.droptable(caslib='public', name='UniProtSTRINGIDs', quiet=True)
    UniProtSTRINGIDs_ct = s.upload_file(data=UniProtSTRINGIDs_df, casout=dict(name='UniProtSTRINGIDs', caslib='public', replication=0, promote=True))
    UniProtSTRINGIDs_ct.info()
    display(UniProtSTRINGIDs_ct.head())


    s.droptable(caslib='public', name='UniProtGeneNames', quiet=True)
    UniProtGeneNames_ct = s.upload_file(data=UniProtGeneNames_df, casout=dict(name='UniProtGeneNames', caslib='public', replication=0, promote=True))
    UniProtGeneNames_ct.info()
    display(UniProtGeneNames_ct.head())

UniProtGeneIDs_df.to_csv(CSVOut+"/"+"UniProtGeneIDs.csv")
UniProtSTRINGIDs_df.to_csv(CSVOut+"/"+"UniProtSTRINGIDs.csv")
UniProtGeneNames_df.to_csv(CSVOut+"/"+"UniProtGeneNames.csv")


In [ ]:
# ClinicalTrials Truth (could possibly benefit from more cleaning):
Truth_df.to_csv(CSVOut+"/"+"Truth.csv")
if casenabled == True:
    s.droptable(caslib='public', name='Truth', quiet=True)
    Truth_ct = s.upload_file(data=Truth_df, casout=dict(name='Truth', caslib='public', replication=0, promote=True, label="Clinical Trials Truth: NOTE needs to be refined"))
    Truth_ct.info()
    display(Truth_ct.head())

# truthMatch2_df has information on the "new" source (non FDA Clinical Trials)
truthMatch2_df.to_csv(CSVOut+"/"+"newTruth.csv")
if casenabled == True:
    s.droptable(caslib='public', name='newTruth', quiet=True)
    newTruth_ct = s.upload_file(data=truthMatch2_df, casout=dict(name='newTruth', caslib='public', replication=0, promote=True, label="Clinical Trials Truth: NOTE needs to be refined"))
    newTruth_ct.info()
    display(newTruth_ct.head())


In [ ]:
# VirusHost Proteins
if casenabled == True:
    s.droptable(caslib='public', name='VirusHostProt', quiet=True)
    VirusHostProt_ct = s.upload_file(data=VirusHostProt_df, casout=dict(name='VirusHostProt', caslib='public', replication=0, promote=True, label="Virus Host Protein Relationships NOTE: Many Proteins had 1+ UniProt IDs"))
    display(VirusHostProt_ct.info())
    display(VirusHostProt_ct.head())

VirusHostProt_df.to_csv(CSVOut+"/"+"VirusHostProt.csv")


In [ ]:
# STRING - protein protein data
if casenabled == True:
    s.droptable(caslib='public', name='stringPP', quiet=True)
    stringPP_ct = s.upload_file(data=stringPP_df, casout=dict(name='stringPP', caslib='public', replication=0, promote=True, label="STRING Protein Protein interactions NOTE Missing UniProt Matches"))
    stringPP_ct.info()
    display(stringPP_ct.head())

stringPP_df.to_csv(CSVOut+"/"+"stringPP.csv")


In [ ]:
#NOTE: indication and description cause data errors when loading to a CASLIB - figure it out later

display(dbSmallM_df[:5])

# clean up drug bank small molecule list and merge in clinical trials "truth" flag
x = dbSmallM_df[['drugbank_id', 'name', 'inchi', 'groups', 'atc_codes', 'categories']]

# replace truth_df with truthMatch2_df
#x = pd.merge(x, Truth_df[['drugbank_id', 'truth']], on='drugbank_id', how='left', indicator=True)
newTruth = truthMatch2_df[['iDBID', 'NewDrug', 'ProteinCount']]
newTruth['truth'] = 1
display(newTruth.head()) 

x = pd.merge(x, newTruth, left_on='drugbank_id', right_on='iDBID', how='left', indicator=True)

display(x['_merge'].value_counts())
x.drop(columns=['_merge'], inplace=True)

x.truth = x.truth.fillna(0)
display(x['truth'].value_counts())


# DrugBank Tables
if casenabled == True:
    s.droptable(caslib='public', name='dbSmallM_Truth', quiet=True)
    #dbSmallM_ct = s.upload_file(data=dbSmallM_df, casout=dict(name='dbSmallM', caslib='public', replication=0, promote=True, label="Drug Bank: Small Molecule drugs (NOTE additional attributes available plus stings need to be parsed)"))
    dbSmallM_ct = s.upload_file(data=x, casout=dict(name='dbSmallM_Truth', caslib='public', replication=0, promote=True, label="Drug Bank: Small Molecule drugs (NOTE additional attributes available plus stings need to be parsed)"))
    dbSmallM_ct.info()
    display(dbSmallM_ct.shape)
    display(dbSmallM_ct.head())

x.to_csv(CSVOut+"/"+"dbSmallM_Truth.csv")


In [ ]:

# drug bank - drug/protein table(s)
if casenabled == True:
    s.droptable(caslib='public', name='dbProteins', quiet=True)
    dbProteins_ct = s.upload_file(data=dbProteins_df, casout=dict(name='dbProteins', caslib='public', replication=0, promote=True, label="Drug Bank: Small Molcule related proteins (NOTE not all captured by XML parser yet)"))
    #dbProteins_ct = s.upload_file(data=dbProteins_df[:5], casout=dict(name='dbProteins', caslib='public', replication=0, promote=True, label="Drug Bank: Small Molcule related proteins"))
    dbProteins_ct.info()
    display(dbProteins_ct.shape)
    display(dbProteins_ct.head())

dbProteins_df.to_csv(CSVOut+"/"+"dbProteins.csv")


3) Join tables and write result to CAS

In [ ]:
#s.close()